# Avaliação Sprint 3

## Import das bibliotecas necessárias

In [46]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pymongo import MongoClient
import json

## Conexão com o banco de dados Mongo

In [58]:
client = MongoClient("localhost", 27017)
artigos_db = client["Artigos"]
artigos_sem_tratamento = artigos_db["Tratamento_1"] #Texto cru, retirado direto do html

## Fazendo o scrapping dos artigos

In [32]:
urls = [
    "https://www1.folha.uol.com.br/banco-de-dados/2021/05/1921-transporte-de-cavalos-de-corrida-burla-proibicao-em-sao-paulo.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/06/1921-apos-sucesso-nos-eua-pianista-guiomar-novaes-chega-a-sao-paulo.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/05/1921-missao-italiana-vem-ao-brasil-buscar-ajuda-para-mutilados-na-guerra.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/05/1971-franca-aprova-entrada-do-reino-unido-no-mercado-comum-europeu.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/05/1921-empresa-de-campinas-vai-comecar-a-vender-carne-fresca-a-paulistanos.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/05/1921-teatro-polytheama-em-ribeirao-preto-e-destruido-por-incendio.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/04/1921-tracado-de-estrada-que-ligara-limeira-a-araras-e-estudado.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/03/1971-guerra-civil-estoura-no-paquistao-oriental-que-deseja-independencia.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/03/1921-estrada-vai-facilitar-ligacao-de-sao-paulo-com-triangulo-mineiro-e-goias.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/02/1921-autoridade-do-reino-unido-reafirma-luta-contra-movimento-irlandes.shtml"
] # lista das urls dos artigos



In [49]:
artigos = []
for url in urls:
    response = urlopen(url)
    html = response.read().decode('utf8')

    soup = BeautifulSoup(html, 'html.parser')

    paragrafos_artigo = soup.find('div', {'class': 'c-news__body'}).findAll('p')
    # Retirando as tags html dos artigos
    paragrafos_artigo = [str(paragrafo).replace('<p>','') for paragrafo in paragrafos_artigo]
    paragrafos_artigo = [str(paragrafo).replace('</p>','') for paragrafo in paragrafos_artigo]

    artigos.append({'texto' : paragrafos_artigo })


In [50]:
artigos[0]

{'texto': ['Está sendo burlada a proibição de transportar animais nas estradas que convergem a São Paulo.',
  'Dias depois do aparecimento da peste bovina nos arredores da capital, as autoridades sanitárias tinham lembrado o governo da necessidade de estabelecer essa proibição.',
  'Apesar da medida, jornais noticiaram a ida e vinda de cavalos de corrida entre a capital e Santos. A fim de mascarar a manobra, os animais são enviados para São Bernardo e de lá são transportados para a outra cidade.',
  'Isso ocorre como se a transmissão da peste não fosse possível nas patas de cavalos ingleses, que custam caro.']}

## Salvando os artigos no MongoDB

In [57]:
for artigo in artigos:
    # Para cada artigo dentro da lista, adiciona no mongo
    artigos_sem_tratamento.insert_one({'Artigo': artigo})